In [6]:
import cn2an
from operation import chinese_to_int

In [7]:

replaceChineseNumbers_test = {
    '一萬八千': 18000,
    '二十三': 23,
    '100,000': 100000,
    '1百萬': 1000000,
    '一百萬': 1000000,
    '一千零一十': 1010,
    '一百零一': 101,
    '二三': 23,
    '九九': 99,
    '3.5萬': 35000,
    "十四": 14,
    "四十": 40,
    "6拾": 60,
    "6拾萬": 600000,
    "拾萬": 100000,
    "10萬": 100000,
    "一百萬": 1000000,
    "100萬": 1000000,
    "拾1": 11,
    "2佰8拾6万3仟": 2863000,
    "三千零八十":3080,
    # "2佰8拾6万3仟8拾": 2863080,
    "2佰8拾6万3仟零8拾": 2863080
}

for key, number in replaceChineseNumbers_test.items():
    key = key.replace(',', '')
    translate_number = cn2an.cn2an(key, 'smart')
    if translate_number != number:
        print(f"錯誤 {key} => {translate_number} (正確答案: {number})")

# 日期

In [26]:
import re
import cn2an

# fact
line_list = [
    # '處拘役零年零月肆拾日', 
    # '處拘役零年零月肆拾日',
    # '有期徒刑零年貳月',
    # "又犯非法製造子彈罪，處有期徒刑壹年，併科罰金新臺幣壹萬元，罰金如易服勞役，以新臺幣壹仟元折算壹日。有期徒刑部分應執行有期徒刑伍年陸月，罰金部分應執行新臺幣參萬伍仟元，罰金如易服勞役，以新臺幣壹仟元折算壹日。",
    # "蕭迪晃駕駛動力交通工具而吐氣所含酒精濃度達每公升零點二五毫克以上，處有期徒刑參月，如易科罰金，以新臺幣壹仟元折算壹日。",
    # "林家緯犯三人以上共同冒用政府機關及公務員名義犯詐欺取財罪，處有期徒刑以壹年肆月。未扣案之林家緯犯罪所得新臺幣貳仟元沒收，於全部或一部不能沒收或不宜執行沒收時，追徵其價額。",
    # "鄭坤盈犯轉讓禁藥罪，累犯，處有期徒刑陸月。又犯如附表所示之罪，各處如附表所示之刑。有期徒刑逾陸月部分，應執行有期徒刑拾壹年。扣案手機壹支（含SIM 卡壹張）、磅秤壹個均沒收。未扣案犯罪所得新臺幣貳萬叁仟壹佰元沒收，於全部或一部不能沒收或不宜執行沒收時，追徵其價額。",
    # "庚○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，處有期徒刑壹年拾月，併科罰金新臺幣貳佰捌拾陸萬參仟捌拾元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。己○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，累犯，處有期徒刑壹年捌月，併科罰金新臺幣貳佰陸拾貳萬肆仟肆佰玖拾元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、 22所示之物，均沒收。丙○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，累犯，處有期徒刑壹年?月，併科罰金新臺幣貳佰參拾捌萬伍仟玖佰元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。甲○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，處有期徒刑柒月，併科罰金新臺幣壹佰貳拾萬元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。",
    # '高榮利犯恐嚇危害安全罪，共貳罪，各處拘役肆拾日，如易科罰金，均以新臺幣壹仟元折算壹日；又犯傷害罪，處拘役伍拾日，如易科罰金，以新臺幣壹仟元折算壹日。應執行拘役壹佰日，如易科罰金，以新臺幣壹仟元折算壹日。',
    # "吳良旭、鄭文旗犯傷害罪，各處拘役貳拾日，如易科罰金，均以新臺幣壹仟元折算壹日；又犯公然侮辱罪，各處拘役壹拾日，如易科罰金，均以新臺幣壹仟元折算壹日。均應執行拘役貳拾伍日，如易科罰金，均以新臺幣壹仟元折算壹日。",
    # "劉福慶幫助犯詐欺取財罪，累犯，處有期徒刑6月，如易科罰金，以新臺幣1千元折算1日。李俊岳幫助犯詐欺取財罪，處有期徒刑5月，如易科罰金，以新臺幣1千元折算1日。陳清溢犯三人以上共同冒用公務員名義詐欺取財罪，處有期徒刑1年8月。偽造之「台北地檢署監管科收據」公文書上偽造之「臺灣臺北地方法院檢察署印」印文1枚沒收。李政哲犯三人以上共同冒用公務員名義詐欺取財罪，處有期徒刑1年4月。緩刑2年，緩刑期內付保護管束，且應於緩刑期間，接受6小時之法治教育課程。偽造之「台北地檢署監管科收據」公文書上偽造之「臺灣臺北地方法院檢察署印」印文1枚沒收。",
    # "黃建偉幫助犯詐欺取財罪，處拘役肆拾伍日，如易科罰金，以新臺幣壹仟元折算壹日。",
    # "簡國峰犯過失傷害罪，處拘役30日，如易科罰金，以新臺幣1千元折算1日。",
    # "陳振昱吐氣所含酒精濃度達每公升零點二五毫克以上駕駛動力交通工具，累犯，處有期徒刑（參月），如易科罰金，以新臺幣壹仟元折算壹日。"
    # "謝式東犯不能安全駕駛動力交通工具罪，累犯，處有期徒陸月，如易科罰金，以新臺幣壹仟元折算壹日。併科新臺幣肆萬元罰金，罰金如易服勞役，以新臺幣壹仟元折算壹日。"
    "許金全犯踰越牆垣侵入住宅竊盜罪，處有徒刑陸月，如易科罰金，以新臺幣壹仟元折算壹日。緩刑貳年，緩刑期間付保護管束，並向指定之政府機關、政府機構、行政法人、社區或其他符合公益目的之機構或團體提供肆拾小時之義務勞務。"
]

total_imprisonment = 0


def parse_time(time_str, line):
    # 解析時間字符串，返回年、月、日的數值
    try:
        return int(cn2an.cn2an(chinese_to_int(time_str) if time_str != '' else 0, 'smart'))
    except ValueError as e:
        print(f"!-> 相關匹配: {time_str}, 錯誤詳情: {e}")
        return 0

for line in line_list:
    
    print("====", line)

    imprisonments = []
    total_imprisonment = 0  

    if '年' in line or '月' in line or '日' in line:
        
        # 合併正則表達式
        pattern = re.compile(r'(有期徒刑|拘役)(?:([\u4e00-\u9fff\d+]{1,2})年)?(?:([\u4e00-\u9fff\d+]{1,2})月)?(?:([\u4e00-\u9fff\d]{1,5})日)?')
        filter_char = {
            '(': '', 
            '（': '', 
            ')': '', 
            '）': '', 
            '以': '', 
            '又': '',
            '月月': '月',
            '萬萬': '萬',
            '年年': '年'
        }
        temp_text = line
        for char in filter_char:
            temp_text = temp_text.replace(char, '')
        
        matches = pattern.findall(temp_text)
        
        for match in matches:
            sentence_type, years, months, days = match

            years_val = parse_time(years, line)
            months_val = parse_time(months, line)
            days_val = parse_time(days, line)

            if years_val != 0 or months_val != 0 or days_val != 0:
                imprisonment_str = f"{sentence_type}{years_val}年{months_val}月{days_val}日" if days else f"{sentence_type}{years_val}年{months_val}月"
                print(imprisonment_str)
            

==== 許金全犯踰越牆垣侵入住宅竊盜罪，處有徒刑陸月，如易科罰金，以新臺幣壹仟元折算壹日。緩刑貳年，緩刑期間付保護管束，並向指定之政府機關、政府機構、行政法人、社區或其他符合公益目的之機構或團體提供肆拾小時之義務勞務。


# 罰金

In [24]:
line_list = [
    # "陳阿順犯森林法第五十二條第三項、第一項第四款、第六款之結夥二人以上為搬運贓物、使用車輛竊取森林主產物貴重木罪，處有期徒刑壹年參月，併科罰金新臺幣貳佰零肆萬玖仟柒佰參拾元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。未扣案如附表所示之犯罪所得均沒收，於全部或一部不能沒收或不宜執行沒收時，各追徵其價額。",
    # "姜瀚承犯非法持有子彈罪，處有期徒刑參月，併科罰金新臺幣壹萬伍仟萬元，有期徒刑如易科罰金、罰金如易服勞役，均以新臺幣壹仟元折算壹日；又犯竊盜未遂罪，處拘役陸日，如易科罰金，以新臺幣壹仟元折算壹日。扣案如附表所示驗餘非制式子彈肆顆均沒收。",
    # "庚○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，處有期徒刑壹年拾月，併科罰金新臺幣貳佰捌拾陸萬參仟捌拾元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。己○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，累犯，處有期徒刑壹年捌月，併科罰金新臺幣貳佰陸拾貳萬肆仟肆佰玖拾元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、 22所示之物，均沒收。丙○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，累犯，處有期徒刑壹年?月，併科罰金新臺幣貳佰參拾捌萬伍仟玖佰元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。甲○○犯森林法第五十二條第三項、第一項第一款、第四款、第六款之竊取森林主產物貴重木罪，處有期徒刑柒月，併科罰金新臺幣壹佰貳拾萬元，罰金如易服勞役，以罰金總額與壹年之日數比例折算。扣案如附表編號7 至14、16至17、22所示之物，均沒收。",
    # "黃建銘竊盜，處罰金新台幣捌仟元，如易服勞役以新台幣壹仟元折算壹日。",
    # "鄭偉雄竊盜，處罰金陸仟元，如易服勞役以新台幣壹仟元折算壹日，緩刑參年。",
    # "甲○○無駕駛執照駕車，因過失傷害人，處拘役伍拾伍日，如易科罰金以新臺幣壹仟元折算壹日。",
    # "徐偉豪犯非法製造可發射子彈具有殺傷力之槍枝罪，處有期徒刑伍年，併科罰金新臺幣參萬元，罰金如易服勞役，以新臺幣壹仟元折算壹日。又犯非法製造子彈罪，處有期徒刑壹年，併科罰金新臺幣壹萬元，罰金如易服勞役，以新臺幣壹仟元折算壹日。有期徒刑部分應執行有期徒刑伍年陸月，罰金部分應執行新臺幣參萬伍仟元，罰金如易服勞役，以新臺幣壹仟元折算壹日。",
    # "罪，處有期徒刑陸月，併科罰金新台幣貳萬元，有期徒刑如易科罰金、罰金如易服勞役均以新臺幣壹仟元折算壹日。"
    "謝式東犯不能安全駕駛動力交通工具罪，累犯，處有期徒陸月，如易科罰金，以新臺幣壹仟元折算壹日。併科新臺幣肆萬元罰金，罰金如易服勞役，以新臺幣壹仟元折算壹日。"
]

pattern = re.compile(r"(?<!易科)(?:罰金(?:新(?:臺|台)幣)?)([\u4e00-\u9fff]+)元(?<!折算壹日)[，。、？]")

for line in line_list:
    print("====", line)
    
    amount_list = []
    matches = pattern.findall(line)  # 使用 findall 來找到所有匹配
    for match in matches:
        try:
            regular_match = cn2an.cn2an(chinese_to_int(match), 'smart')
            print(f"O -> 相關匹配: {regular_match}")
        except Exception as e:
            print(f"!error -> 相關匹配: {match}, 錯誤詳情: {e}")


==== 謝式東犯不能安全駕駛動力交通工具罪，累犯，處有期徒陸月，如易科罰金，以新臺幣壹仟元折算壹日。併科新臺幣肆萬元罰金，罰金如易服勞役，以新臺幣壹仟元折算壹日。


In [4]:
import re
def re_charges(content, pattern=r'裁判字號：(.+)'):
    match = re.search(pattern, content, re.MULTILINE)
    if match:
        return match.group(1).strip()
    return ""

# 使用範例
content = "\n".join(["裁判字號：臺灣彰化地方法院 110 年度 交易 字第 268 號 判決"])
charges = re_charges(content)
print(charges)

臺灣彰化地方法院 110 年度 交易 字第 268 號 判決


In [23]:

import re
def re_charges(content, pattern=r'裁判案由：(.+)'):
    for line in content:
        match = re.search(pattern, line, re.MULTILINE)
        if match:
            return match.group(1).strip()
    return ""

# 使用範例
content = [
        "裁判字號：臺灣彰化地方法院 110 年度 交易 字第 268 號 判決",
        "裁判案由：過失傷害",
        ".......",
        "...."
]

charges = re_charges(content)
print(charges)

過失傷害
